In [1]:
import os
import numpy as np 
import scipy.io.wavfile as wav 
import python_speech_features as psf
import hmmlearn.hmm as hmm
import librosa

In [2]:
def extract_mfcc_features(filename):
    # load the wave file and get the sample rate and audio data
    sample_rate, audio = wav.read(filename)
    # extract mfcc features
    # change sample rate
    # sample_rate = librosa.resample(y=audio, orig_sr=sample_rate, target_sr=16000)
    # audio = audio.astype(np.float64) / np.iinfo(audio.dtype).max
    mfcc_features = psf.mfcc(audio, samplerate=16000)
    return mfcc_features

In [3]:
extract_mfcc_features("/home/ix502iv/Documents/Audio_Trad/DTW/Lecture/audio/cat.wav")

array([[ 11.23534804, -13.08661709,   5.75003537, ...,  -0.3884813 ,
          2.09732568,  -3.36208155],
       [ 10.6868725 , -18.82335808,  12.21479618, ...,   2.22484038,
          4.46145968,  -4.37608331],
       [ 10.61177596, -19.11138554,   9.59082019, ...,  -6.3604963 ,
          5.25303554,   2.65838198],
       ...,
       [ 11.2586154 ,  -8.33601334,   2.78819231, ...,  -5.28767173,
          6.66102993,  14.31883911],
       [ 11.06969225, -12.67100128,  -0.55847585, ..., -16.24812375,
         -3.79298751,   7.33667571],
       [ 10.54895955, -11.97425769,   1.53086278, ..., -13.47627786,
         -6.70694104,   4.75390341]])

In [4]:
# load the .wav file for each word in the training set
word_directories = ["../Day_2/Gen/1/","../Day_2/Gen/2/","../Day_2/Gen/3/",
                    "../Day_2/Gen/4/", "../Day_2/Gen/5/"] # directories where each of the word is contained
word_models = [] # initialize an empty list where we will store trained models

for word_directory in word_directories:
    word_mfcc_features = []
    for filename in os.listdir(word_directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(word_directory, filename)
            word_mfcc_features.append(extract_mfcc_features(filepath))

    # stack mfcc features for all audio files in the directory
    word_mfcc_features = np.vstack(word_mfcc_features)
    
    # train an HMM model on the MFCC features
    num_states = 5 # hidden number of states.
    num_mixtures = 13 # btn 8 and 16

    # specify the starting probs
    start_prob = np.array([0.1, 0.1, 0.1, 0.1, 0.6])

    word_model = hmm.GMMHMM(n_components=num_states, n_mix=num_mixtures, startprob_prior=start_prob)
    word_model.fit(word_mfcc_features)
    word_models.append(word_model)

/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/utils.py:29: RuntimeWarning: invalid value encountered in divide
  a /= a_sum


In [5]:
def validate_model(word_mfcc_features, word_models):
    log_likelihood = np.zeros(len(word_models))

    for i, model in enumerate(word_models):
        log_likelihood[i] = model.score(word_mfcc_features)

    # predict the word with highest likelihood
    word_index = np.argmax(log_likelihood)
    return word_index

In [6]:
word_index= validate_model(word_mfcc_features, word_models)

Degenerate mixture covariance


ValueError: startprob_ must sum to 1 (got nan)